Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [20]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print(hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 7, 7, 16)
(16, 784)
(10000, 7, 7, 16)
(10000, 784)
(10000, 7, 7, 16)
(10000, 784)


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.907547
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.522375
Minibatch accuracy: 37.5%
Validation accuracy: 67.3%
Minibatch loss at step 100: 0.816986
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 150: 1.427509
Minibatch accuracy: 62.5%
Validation accuracy: 75.7%
Minibatch loss at step 200: 0.394903
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 250: 0.847563
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 300: 1.028584
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 350: 0.688859
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.545253
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 450: 0.661375
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.741445
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    max_pool = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer1_biases)
    max_pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.486484
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.316186
Minibatch accuracy: 12.5%
Validation accuracy: 17.7%
Minibatch loss at step 100: 2.274845
Minibatch accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss at step 150: 2.215283
Minibatch accuracy: 18.8%
Validation accuracy: 13.7%
Minibatch loss at step 200: 2.086627
Minibatch accuracy: 18.8%
Validation accuracy: 19.4%
Minibatch loss at step 250: 2.347556
Minibatch accuracy: 12.5%
Validation accuracy: 18.0%
Minibatch loss at step 300: 1.884783
Minibatch accuracy: 37.5%
Validation accuracy: 36.2%
Minibatch loss at step 350: 2.045194
Minibatch accuracy: 25.0%
Validation accuracy: 33.2%
Minibatch loss at step 400: 2.119556
Minibatch accuracy: 25.0%
Validation accuracy: 27.8%
Minibatch loss at step 450: 1.472070
Minibatch accuracy: 62.5%
Validation accuracy: 30.0%
Minibatch loss at step 500: 1.597845
Minibatch accuracy: 43.8%
Validation accuracy: 52.2%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [37]:
batch_size = 2000
patch_size = 5
depth = 16
num_hidden = 64
stddev = 0.1

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.  
  conv1_weights = tf.Variable(tf.truncated_normal([1, 1, 1, 4], stddev=stddev))
  conv1_biases = tf.Variable(tf.constant(0.001, shape=[4]))

  conv2_weights = tf.Variable(tf.truncated_normal([1, 1, 1, 4], stddev=stddev))
  conv2_biases = tf.Variable(tf.constant(0.001, shape=[4]))

  conv3_weights = tf.Variable(tf.truncated_normal([1, 1, 8, 8], stddev=stddev))
  conv3_biases = tf.Variable(tf.constant(0.001, shape=[8]))

  conv4_weights = tf.Variable(tf.truncated_normal([2, 2, 8, 8], stddev=stddev))
  conv4_biases = tf.Variable(tf.constant(0.001, shape=[8]))

  conv5_weights = tf.Variable(tf.truncated_normal([1, 1, 16, 16], stddev=stddev))
  conv5_biases = tf.Variable(tf.constant(0.001, shape=[16]))
  
  conv6_weights = tf.Variable(tf.truncated_normal([3, 3, 16, 16], stddev=stddev))
  conv6_biases = tf.Variable(tf.constant(0.001, shape=[16]))

  fc1_weights = tf.Variable(tf.truncated_normal([7 * 7 * 32, 512], stddev=stddev))
  fc1_biases = tf.Variable(tf.constant(0.001, shape=[512]))

  fc2_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=stddev))
  fc2_biases = tf.Variable(tf.constant(0.001, shape=[num_labels]))

  # Model.
  def model(data, with_dropout=False):
    conv1 = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
    conv1 = tf.nn.relu(conv1 + conv1_biases)
    
    pool1 = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.conv2d(pool1, conv2_weights, [1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv2 + conv2_biases)
    
    miniception1 = tf.concat(3, [conv1, conv2])
    
    conv3 = tf.nn.conv2d(miniception1, conv3_weights, [1, 1, 1, 1], padding='SAME')
    conv3 = tf.nn.relu(conv3 + conv3_biases)
    
    pool2 = tf.nn.max_pool(miniception1, [1, 2, 2, 1], [1, 1, 1, 1], padding='SAME')
    conv4 = tf.nn.conv2d(pool2, conv4_weights, [1, 1, 1, 1], padding='SAME')
    conv4 = tf.nn.relu(conv4 + conv4_biases)
    
    miniception2 = tf.concat(3, [conv3, conv4])
    
    pool3 = tf.nn.max_pool(miniception2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv5 = tf.nn.conv2d(pool3, conv5_weights, [1, 1, 1, 1], padding='SAME')
    conv5 = tf.nn.relu(conv5 + conv5_biases)
    
    conv6 = tf.nn.conv2d(pool3, conv6_weights, [1, 1, 1, 1], padding='SAME')
    conv6 = tf.nn.relu(conv6 + conv6_biases)
    
    miniception3 = tf.concat(3, [conv5, conv6])
    
    pool4 = tf.nn.max_pool(miniception3, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    # Bridge from 3D to 1D
    final_filter = pool4
    shape = final_filter.get_shape().as_list()
    bridged = tf.reshape(final_filter, [shape[0], shape[1] * shape[2] * shape[3]])
    
    fc1 = tf.matmul(bridged, fc1_weights) + fc1_biases
    if with_dropout:
        fc1 = tf.nn.dropout(fc1, 0.5)
    fc1 = tf.nn.relu(fc1)

    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases
    if with_dropout:
        fc2 = tf.nn.dropout(fc2, 0.5)
    fc2 = tf.nn.relu(fc2)
    
    final = fc2
    return final

  # Training computation.
  logits = model(tf_train_dataset, with_dropout=True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(model(tf_train_dataset))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 501

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.304138
Minibatch accuracy: 8.5%
Validation accuracy: 11.6%
Minibatch loss at step 50: 2.295171
Minibatch accuracy: 27.9%
Validation accuracy: 29.3%
Minibatch loss at step 100: 2.271788
Minibatch accuracy: 44.2%
Validation accuracy: 38.9%
Minibatch loss at step 150: 2.220069
Minibatch accuracy: 50.1%
Validation accuracy: 50.9%
Minibatch loss at step 200: 2.085104
Minibatch accuracy: 56.5%
Validation accuracy: 56.6%
Minibatch loss at step 250: 1.942979
Minibatch accuracy: 64.9%
Validation accuracy: 65.0%
Minibatch loss at step 300: 1.820863
Minibatch accuracy: 69.7%
Validation accuracy: 69.8%
Minibatch loss at step 350: 1.787056
Minibatch accuracy: 69.1%
Validation accuracy: 71.0%
Minibatch loss at step 400: 1.727050
Minibatch accuracy: 71.5%
Validation accuracy: 72.3%
Minibatch loss at step 450: 1.678516
Minibatch accuracy: 73.1%
Validation accuracy: 72.7%
Minibatch loss at step 500: 1.640676
Minibatch accuracy: 80.3%
Validation accuracy: 80.9%
Te